Action1	使用Wide&Deep模型对movielens进行评分预测

In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
from deepctr.models import WDL

In [6]:
from deepctr.feature_column import SparseFeat,get_feature_names

In [7]:
data = pd.read_csv("movielens_sample.txt")

In [10]:
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,107,12,4,968035345,Ed Wood (1994),Comedy|Drama,0,2,4,35
1,123,169,3,966536874,Patriot Games (1992),Action|Thriller,1,1,4,118
2,12,6,4,976203603,"Bridges of Madison County, The (1995)",Drama|Romance,0,2,13,99
3,21,112,3,975430389,Indiana Jones and the Temple of Doom (1984),Action|Adventure,1,1,18,55
4,187,45,5,957782527,"Apartment, The (1960)",Comedy|Drama,1,5,19,41


In [8]:
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ['rating']

In [9]:
# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])

In [12]:
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
fixlen_feature_columns

[SparseFeat(name='movie_id', vocabulary_size=187, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x7f8c175b19d0>, embedding_name='movie_id', group_name='default_group', trainable=True),
 SparseFeat(name='user_id', vocabulary_size=193, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x7f8c8419c4d0>, embedding_name='user_id', group_name='default_group', trainable=True),
 SparseFeat(name='gender', vocabulary_size=2, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x7f8c841a8690>, embedding_name='gender', group_name='default_group', trainable=True),
 SparseFeat(name='age', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x7f8c175b1fd0>, emb

In [15]:
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns

In [16]:
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [17]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [25]:
train_model_input

{'movie_id': array([173,  68, 158,  93, 162,  25,   3,  66,  70, 144, 113,  15,  85,
        157, 186, 123, 112, 130,   9, 180, 100,   2, 154,  11,  84,  97,
        161,  21,  49,  41,  13, 108,  69,  94, 132, 136, 181,  83, 173,
         99,  23,  17,  30,  27,  86,  72,  34,  35, 121,  32, 183,  55,
        120,  96,  59,  50, 163, 123,  77,  28,  57, 153,  51, 149, 164,
         90,  72, 184,  26,  80,  79,  52, 111, 128,  48,  62,  64,  60,
         43,  87, 182,  40,  78, 140,  46, 169,  18, 125, 103,  20, 142,
        147, 102,  27,  19, 160, 143,  63, 118, 106,  16, 148,  29,  10,
          4,  73, 166, 156, 178,  89,  45, 150,   5, 174,  91, 109,   0,
        133,  47, 159,  67,  39, 117, 149,   6, 176, 172, 171, 168, 137,
         42,  82,  12,  44,  95,  38,  22, 119, 115, 126,  24,  74,  14,
        138,  54, 169,   8, 101, 170,  92,  33,  36, 116, 114,   7,  34,
         76, 105, 177,  71]),
 'user_id': array([ 10, 161,  49,  40,  44,   9,  71, 183,  72,  64, 188, 181,  84

In [18]:
# 使用WDL进行训练
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1/1 [==============================] - 2s 2s/step - loss: 14.5233 - mse: 14.5233 - val_loss: 12.7351 - val_mse: 12.7351


In [19]:
# 使用WDL进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

test RMSE 3.7192875661879117
